In [1]:
from bs4 import BeautifulSoup
import requests

from IPython.core.display import display, HTML

import pandas as pd

import re

import numpy as np

import datetime

In [2]:
#Retrieve breakdown of games from october to end of march

urlList = [
    'https://www.basketball-reference.com/leagues/NBA_2017_games-october.html',
    'https://www.basketball-reference.com/leagues/NBA_2017_games-november.html',
    'https://www.basketball-reference.com/leagues/NBA_2017_games-december.html',
    'https://www.basketball-reference.com/leagues/NBA_2017_games-january.html',
    'https://www.basketball-reference.com/leagues/NBA_2017_games-february.html',
    'https://www.basketball-reference.com/leagues/NBA_2017_games-march.html'
]
soupList = []
for url in urlList:
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    soupList.append(soup)
[soup1,soup2,soup3,soup4,soup5,soup6] = soupList

In [3]:
headerData = soup1.find(class_ = 'overthrow table_container').find_all('tr')[0].find_all('th')

In [4]:
column_headers = []

#get headers
for header in headerData:
    column_headers.append(header['data-stat'])
column_headers

['date_game',
 'game_start_time',
 'visitor_team_name',
 'visitor_pts',
 'home_team_name',
 'home_pts',
 'box_score_text',
 'overtimes',
 'attendance',
 'game_remarks']

In [5]:
fullDataList = []

for soup in soupList:
    cellData = soup.find(class_ = 'overthrow table_container').find_all('tr')
    for row in range(1,len(cellData)):
        rowList = []
        for cell in cellData[row].find_all(lambda tag: tag.name == 'td' or tag.name == 'th'):
            if cell.contents == []:
                value = ''
            elif cell.contents[0].name == 'a':
                if cell.contents[0].contents[0] == 'Box Score':
                    value = 'https://www.basketball-reference.com'+ cell.contents[0]['href']
                else:
                    value = cell.contents[0].contents[0]
            else:
                value = cell.contents[0]
            rowList.append(value)
        fullDataList.append(rowList)

cleanCellData = [x for x in fullDataList if x != []]

cleanCellData

[['Tue, Oct 25, 2016',
  '7:30p',
  'New York Knicks',
  '88',
  'Cleveland Cavaliers',
  '117',
  'https://www.basketball-reference.com/boxscores/201610250CLE.html',
  '',
  '20,562',
  ''],
 ['Tue, Oct 25, 2016',
  '10:00p',
  'Utah Jazz',
  '104',
  'Portland Trail Blazers',
  '113',
  'https://www.basketball-reference.com/boxscores/201610250POR.html',
  '',
  '19,446',
  ''],
 ['Tue, Oct 25, 2016',
  '10:30p',
  'San Antonio Spurs',
  '129',
  'Golden State Warriors',
  '100',
  'https://www.basketball-reference.com/boxscores/201610250GSW.html',
  '',
  '19,596',
  ''],
 ['Wed, Oct 26, 2016',
  '7:00p',
  'Dallas Mavericks',
  '121',
  'Indiana Pacers',
  '130',
  'https://www.basketball-reference.com/boxscores/201610260IND.html',
  'OT',
  '17,923',
  ''],
 ['Wed, Oct 26, 2016',
  '7:00p',
  'Miami Heat',
  '108',
  'Orlando Magic',
  '96',
  'https://www.basketball-reference.com/boxscores/201610260ORL.html',
  '',
  '19,298',
  ''],
 ['Wed, Oct 26, 2016',
  '7:30p',
  'Brooklyn N

In [6]:
#Combine Header and Data to create intiial DF

df = pd.DataFrame(cleanCellData,columns = column_headers)

In [7]:
def date_change(row):
    t = datetime.datetime.strptime(row['date_game'].replace(',',''), "%a %b %d %Y")
    convert_date = t.strftime('%m/%d/%Y')
    return(convert_date)

df['Date'] = pd.to_datetime(df.apply(date_change,axis = 1))

In [8]:
nbaDict = {
'Atlanta Hawks': 'ATL',
'Brooklyn Nets': 'BRK',
'Boston Celtics': 'BOS',
'Charlotte Hornets': 'CHO',
'Chicago Bulls': 'CHI',
'Cleveland Cavaliers': 'CLE',
'Dallas Mavericks': 'DAL',
'Denver Nuggets': 'DEN',
'Detroit Pistons': 'DET',
'Golden State Warriors': 'GSW',
'Houston Rockets': 'HOU',
'Indiana Pacers': 'IND',
'Los Angeles Clippers': 'LAC',
'Los Angeles Lakers': 'LAL',
'Memphis Grizzlies': 'MEM',
'Miami Heat': 'MIA',
'Milwaukee Bucks': 'MIL',
'Minnesota Timberwolves': 'MIN',
'New Orleans Pelicans': 'NOP',
'New York Knicks': 'NYK',
'Oklahoma City Thunder': 'OKC',
'Orlando Magic': 'ORL',
'Philadelphia 76ers': 'PHI',
'Phoenix Suns': 'PHO',
'Portland Trail Blazers': 'POR',
'Sacramento Kings': 'SAC',
'San Antonio Spurs': 'SAS',
'Toronto Raptors': 'TOR',
'Utah Jazz': 'UTA',
'Washington Wizards': 'WAS'}

In [9]:
statColumns = ['Team TS%','Team eFG%','Team ORtg','Opp TS%','Opp eFG%','Opp ORtg']

In [10]:
def statRecorder(row):
    url1 = row['box_score_text']
    response1 = requests.get(url1)
    page1 = response1.text
    soup1 = BeautifulSoup(page1, "lxml")
    team_list = [nbaDict[row['visitor_team_name']],nbaDict[row['home_team_name']]]
    print([row['visitor_team_name'],row['home_team_name']])
    statsList = []
    index = [1,2,-2]
    for team in team_list:
        boxScoreLink = "box-{}-game-advanced".format(team)
        totalStats = soup1.find('table', id =boxScoreLink).find_all('tr')[-1].find_all('td')
        for i in index:
            statsList.append(totalStats[i].contents[0])
    return(statsList)

In [11]:
#Create new stats list incrementally and then concatenate after
stats_list = []

In [12]:
firstBatch = df[0:1200].apply(statRecorder,axis = 1)
for i in firstBatch:
    stats_list.append(i)

['New York Knicks', 'Cleveland Cavaliers']
['Utah Jazz', 'Portland Trail Blazers']
['San Antonio Spurs', 'Golden State Warriors']
['Dallas Mavericks', 'Indiana Pacers']
['Miami Heat', 'Orlando Magic']
['Brooklyn Nets', 'Boston Celtics']
['Detroit Pistons', 'Toronto Raptors']
['Denver Nuggets', 'New Orleans Pelicans']
['Charlotte Hornets', 'Milwaukee Bucks']
['Minnesota Timberwolves', 'Memphis Grizzlies']
['Oklahoma City Thunder', 'Philadelphia 76ers']
['Sacramento Kings', 'Phoenix Suns']
['Houston Rockets', 'Los Angeles Lakers']
['Washington Wizards', 'Atlanta Hawks']
['Boston Celtics', 'Chicago Bulls']
['Los Angeles Clippers', 'Portland Trail Blazers']
['San Antonio Spurs', 'Sacramento Kings']
['Cleveland Cavaliers', 'Toronto Raptors']
['Indiana Pacers', 'Brooklyn Nets']
['Orlando Magic', 'Detroit Pistons']
['Phoenix Suns', 'Oklahoma City Thunder']
['Charlotte Hornets', 'Miami Heat']
['Houston Rockets', 'Dallas Mavericks']
['Los Angeles Lakers', 'Utah Jazz']
['Golden State Warriors', 

['Miami Heat', 'Philadelphia 76ers']
['Phoenix Suns', 'Washington Wizards']
['Houston Rockets', 'Detroit Pistons']
['Orlando Magic', 'Milwaukee Bucks']
['Boston Celtics', 'Minnesota Timberwolves']
['Dallas Mavericks', 'San Antonio Spurs']
['Toronto Raptors', 'Los Angeles Clippers']
['New Orleans Pelicans', 'Atlanta Hawks']
['Portland Trail Blazers', 'New York Knicks']
['Chicago Bulls', 'Denver Nuggets']
['Oklahoma City Thunder', 'Los Angeles Lakers']
['San Antonio Spurs', 'Charlotte Hornets']
['Portland Trail Blazers', 'Cleveland Cavaliers']
['Atlanta Hawks', 'Indiana Pacers']
['Phoenix Suns', 'Orlando Magic']
['Memphis Grizzlies', 'Philadelphia 76ers']
['Boston Celtics', 'Brooklyn Nets']
['Miami Heat', 'Detroit Pistons']
['Toronto Raptors', 'Houston Rockets']
['Los Angeles Clippers', 'Dallas Mavericks']
['Denver Nuggets', 'Utah Jazz']
['Minnesota Timberwolves', 'New Orleans Pelicans']
['Los Angeles Lakers', 'Golden State Warriors']
['Oklahoma City Thunder', 'Sacramento Kings']
['San A

['Los Angeles Lakers', 'Cleveland Cavaliers']
['Houston Rockets', 'Minnesota Timberwolves']
['New York Knicks', 'Denver Nuggets']
['Portland Trail Blazers', 'Golden State Warriors']
['Los Angeles Clippers', 'Washington Wizards']
['Sacramento Kings', 'Dallas Mavericks']
['Utah Jazz', 'Memphis Grizzlies']
['Boston Celtics', 'Miami Heat']
['Toronto Raptors', 'Orlando Magic']
['Brooklyn Nets', 'Philadelphia 76ers']
['New Orleans Pelicans', 'San Antonio Spurs']
['Washington Wizards', 'Indiana Pacers']
['Detroit Pistons', 'Chicago Bulls']
['Phoenix Suns', 'Minnesota Timberwolves']
['Atlanta Hawks', 'Oklahoma City Thunder']
['Dallas Mavericks', 'Denver Nuggets']
['Los Angeles Lakers', 'Charlotte Hornets']
['New Orleans Pelicans', 'Philadelphia 76ers']
['Orlando Magic', 'Miami Heat']
['Indiana Pacers', 'New York Knicks']
['Brooklyn Nets', 'Toronto Raptors']
['San Antonio Spurs', 'Houston Rockets']
['Boston Celtics', 'Memphis Grizzlies']
['Cleveland Cavaliers', 'Milwaukee Bucks']
['Utah Jazz', 

['Los Angeles Lakers', 'Los Angeles Clippers']
['New Orleans Pelicans', 'Chicago Bulls']
['San Antonio Spurs', 'Phoenix Suns']
['Philadelphia 76ers', 'Washington Wizards']
['Orlando Magic', 'Utah Jazz']
['Minnesota Timberwolves', 'Dallas Mavericks']
['Milwaukee Bucks', 'Atlanta Hawks']
['New York Knicks', 'Toronto Raptors']
['Houston Rockets', 'Brooklyn Nets']
['Chicago Bulls', 'Memphis Grizzlies']
['Oklahoma City Thunder', 'Sacramento Kings']
['Detroit Pistons', 'Los Angeles Lakers']
['Atlanta Hawks', 'New York Knicks']
['Portland Trail Blazers', 'Washington Wizards']
['Philadelphia 76ers', 'Milwaukee Bucks']
['New Orleans Pelicans', 'Indiana Pacers']
['Orlando Magic', 'Denver Nuggets']
['Charlotte Hornets', 'Boston Celtics']
['Cleveland Cavaliers', 'Golden State Warriors']
['Utah Jazz', 'Phoenix Suns']
['Oklahoma City Thunder', 'Los Angeles Clippers']
['Toronto Raptors', 'Brooklyn Nets']
['Houston Rockets', 'Miami Heat']
['Dallas Mavericks', 'Chicago Bulls']
['Minnesota Timberwolves'

['San Antonio Spurs', 'Detroit Pistons']
['Denver Nuggets', 'New York Knicks']
['Golden State Warriors', 'Memphis Grizzlies']
['Los Angeles Lakers', 'Milwaukee Bucks']
['New Orleans Pelicans', 'Minnesota Timberwolves']
['Indiana Pacers', 'Washington Wizards']
['Chicago Bulls', 'Phoenix Suns']
['Atlanta Hawks', 'Sacramento Kings']
['Los Angeles Clippers', 'Charlotte Hornets']
['Milwaukee Bucks', 'Indiana Pacers']
['Denver Nuggets', 'Cleveland Cavaliers']
['Miami Heat', 'Philadelphia 76ers']
['Golden State Warriors', 'Oklahoma City Thunder']
['Orlando Magic', 'Dallas Mavericks']
['Phoenix Suns', 'Houston Rockets']
['Boston Celtics', 'Utah Jazz']
['Chicago Bulls', 'Minnesota Timberwolves']
['San Antonio Spurs', 'New York Knicks']
['Detroit Pistons', 'Toronto Raptors']
['New Orleans Pelicans', 'Sacramento Kings']
['Philadelphia 76ers', 'Charlotte Hornets']
['San Antonio Spurs', 'Indiana Pacers']
['Memphis Grizzlies', 'Brooklyn Nets']
['Orlando Magic', 'Miami Heat']
['Detroit Pistons', 'Mil

['Indiana Pacers', 'New York Knicks']
['Portland Trail Blazers', 'New Orleans Pelicans']
['Philadelphia 76ers', 'Golden State Warriors']
['Charlotte Hornets', 'Indiana Pacers']
['Dallas Mavericks', 'Washington Wizards']
['Minnesota Timberwolves', 'Boston Celtics']
['Utah Jazz', 'Detroit Pistons']
['New Orleans Pelicans', 'Miami Heat']
['Memphis Grizzlies', 'Chicago Bulls']
['Los Angeles Lakers', 'Houston Rockets']
['Portland Trail Blazers', 'San Antonio Spurs']
['Sacramento Kings', 'Phoenix Suns']
['Milwaukee Bucks', 'Los Angeles Clippers']
['Utah Jazz', 'Cleveland Cavaliers']
['Oklahoma City Thunder', 'Toronto Raptors']
['Memphis Grizzlies', 'Atlanta Hawks']
['Brooklyn Nets', 'New York Knicks']
['Los Angeles Clippers', 'Denver Nuggets']
['Orlando Magic', 'Golden State Warriors']
['Dallas Mavericks', 'Philadelphia 76ers']
['Chicago Bulls', 'Washington Wizards']
['Boston Celtics', 'Brooklyn Nets']
['Toronto Raptors', 'Detroit Pistons']
['Minnesota Timberwolves', 'Miami Heat']
['Houston 

In [13]:
statsdf = pd.DataFrame(stats_list,columns = statColumns)

In [14]:
resultdf = pd.concat([df, statsdf], axis=1)
resultdf.head()

,date_game,game_start_time,visitor_team_name,visitor_pts,home_team_name,home_pts,box_score_text,overtimes,attendance,game_remarks,Date,Team TS%,Team eFG%,Team ORtg,Opp TS%,Opp eFG%,Opp ORtg
0,"Tue, Oct 25, 2016",7:30p,New York Knicks,88,Cleveland Cavaliers,117,https://www.basketball-reference.com/boxscores...,,"20,562",,2016-10-25,.459,.420,88.1,.572,.548,117.1
1,"Tue, Oct 25, 2016",10:00p,Utah Jazz,104,Portland Trail Blazers,113,https://www.basketball-reference.com/boxscores...,,"19,446",,2016-10-25,.584,.537,114.9,.667,.607,124.8
2,"Tue, Oct 25, 2016",10:30p,San Antonio Spurs,129,Golden State Warriors,100,https://www.basketball-reference.com/boxscores...,,"19,596",,2016-10-25,.589,.541,131.3,.538,.512,101.8
3,"Wed, Oct 26, 2016",7:00p,Dallas Mavericks,121,Indiana Pacers,130,https://www.basketball-reference.com/boxscores...,OT,"17,923",,2016-10-26,.541,.519,105.8,.602,.559,113.7
4,"Wed, Oct 26, 2016",7:00p,Miami Heat,108,Orlando Magic,96,https://www.basketball-reference.com/boxscores...,,"19,298",,2016-10-26,.519,.505,114.5,.474,.416,101.8


In [15]:
#visitor stats
visitor_df = resultdf.iloc[:,np.r_[0:7,11:14]]
visitor_df.head()

#home stats
home_df = resultdf.iloc[:,np.r_[0:2,4:6,2:4,6,14:17]]
home_df.head()

visitor_df.head()


,date_game,game_start_time,visitor_team_name,visitor_pts,home_team_name,home_pts,box_score_text,Team TS%,Team eFG%,Team ORtg
0,"Tue, Oct 25, 2016",7:30p,New York Knicks,88,Cleveland Cavaliers,117,https://www.basketball-reference.com/boxscores...,.459,.420,88.1
1,"Tue, Oct 25, 2016",10:00p,Utah Jazz,104,Portland Trail Blazers,113,https://www.basketball-reference.com/boxscores...,.584,.537,114.9
2,"Tue, Oct 25, 2016",10:30p,San Antonio Spurs,129,Golden State Warriors,100,https://www.basketball-reference.com/boxscores...,.589,.541,131.3
3,"Wed, Oct 26, 2016",7:00p,Dallas Mavericks,121,Indiana Pacers,130,https://www.basketball-reference.com/boxscores...,.541,.519,105.8
4,"Wed, Oct 26, 2016",7:00p,Miami Heat,108,Orlando Magic,96,https://www.basketball-reference.com/boxscores...,.519,.505,114.5


In [16]:
home_df.columns = ['date_game','game_start_time','team_name','team_pts','opposing_team','opposing_pts','box_score','TS%','eFG%','ORtg']

visitor_df.columns = ['date_game','game_start_time','team_name','team_pts','opposing_team','opposing_pts','box_score','TS%','eFG%','ORtg']


In [17]:
frames = [visitor_df, home_df]
agg_data = pd.concat(frames,ignore_index=True)

In [18]:
def date_change(row):
    t = datetime.datetime.strptime(row['date_game'].replace(',',''), "%a %b %d %Y")
    convert_date = t.strftime('%m/%d/%Y')
    return(convert_date)

agg_data['Date'] = pd.to_datetime(agg_data.apply(date_change,axis = 1))

In [19]:
agg_data['date_game'] = agg_data['Date']
del agg_data['Date']

In [20]:
ordered_agg_data = agg_data.sort_values(by = ['team_name','date_game']).reset_index()

In [21]:
rolling_stat_data = agg_data.sort_values(by = ['team_name','date_game'])

testdata_grouped_rolling = rolling_stat_data.groupby('team_name')[['TS%','eFG%','ORtg']].rolling(window=30, min_periods=1).mean().reset_index()



In [22]:
del testdata_grouped_rolling['level_1']
del testdata_grouped_rolling['team_name']

In [23]:
testdata_grouped_rolling.columns = ['TS1%','eFG1%','ORtg1']

In [24]:
full_stats_data = pd.concat([ordered_agg_data ,testdata_grouped_rolling], axis=1)

In [25]:
#Shift average stats so each row has the average of the games previously
full_stats_data[['TS%','eFG%','ORtg']] = full_stats_data.groupby('team_name')['TS1%','eFG1%','ORtg1'].apply(lambda grp: grp.shift(1))

In [26]:
#Deleted non-shifted columns

full_stats_data = full_stats_data.drop(['TS1%','eFG1%','ORtg1'],axis =1)

In [27]:
full_stats_data_2 = full_stats_data

In [28]:
newDf = pd.merge(full_stats_data_2,full_stats_data,left_on = ['date_game','team_name'],right_on = ['date_game','opposing_team'])

In [29]:
dedupe_new_df = newDf.drop_duplicates(subset = 'box_score_x',keep ='first')

In [30]:
dedupe_new_df.head()

,index_x,date_game,game_start_time_x,team_name_x,team_pts_x,opposing_team_x,opposing_pts_x,box_score_x,TS%_x,eFG%_x,...,index_y,game_start_time_y,team_name_y,team_pts_y,opposing_team_y,opposing_pts_y,box_score_y,TS%_y,eFG%_y,ORtg_y
0,1148,2016-10-27,7:30p,Atlanta Hawks,114,Washington Wizards,99,https://www.basketball-reference.com/boxscores...,NaN,NaN,...,13,7:30p,Washington Wizards,99,Atlanta Hawks,114,https://www.basketball-reference.com/boxscores...,NaN,NaN,NaN
1,25,2016-10-29,12:30p,Atlanta Hawks,104,Philadelphia 76ers,72,https://www.basketball-reference.com/boxscores...,0.594000,0.568000,...,1160,12:30p,Philadelphia 76ers,72,Atlanta Hawks,104,https://www.basketball-reference.com/boxscores...,0.51100,0.470000,96.700000
2,1176,2016-10-31,7:30p,Atlanta Hawks,106,Sacramento Kings,95,https://www.basketball-reference.com/boxscores...,0.570500,0.546000,...,41,7:30p,Sacramento Kings,95,Atlanta Hawks,106,https://www.basketball-reference.com/boxscores...,0.55800,0.500667,112.866667
3,1191,2016-11-02,7:30p,Atlanta Hawks,116,Los Angeles Lakers,123,https://www.basketball-reference.com/boxscores...,0.557667,0.526333,...,56,7:30p,Los Angeles Lakers,123,Atlanta Hawks,116,https://www.basketball-reference.com/boxscores...,0.52925,0.483500,103.225000
4,69,2016-11-04,7:00p,Atlanta Hawks,92,Washington Wizards,95,https://www.basketball-reference.com/boxscores...,0.566500,0.531500,...,1204,7:00p,Washington Wizards,95,Atlanta Hawks,92,https://www.basketball-reference.com/boxscores...,0.54700,0.510667,102.166667


In [31]:
dedupe_new_df.columns

Index(['index_x', 'date_game', 'game_start_time_x', 'team_name_x',
       'team_pts_x', 'opposing_team_x', 'opposing_pts_x', 'box_score_x',
       'TS%_x', 'eFG%_x', 'ORtg_x', 'index_y', 'game_start_time_y',
       'team_name_y', 'team_pts_y', 'opposing_team_y', 'opposing_pts_y',
       'box_score_y', 'TS%_y', 'eFG%_y', 'ORtg_y'],
      dtype='object')

In [32]:
allowed_df = dedupe_new_df[['date_game','team_name_x','opposing_team_x','TS%_x','eFG%_x', 'ORtg_x','TS%_y', 'eFG%_y', 'ORtg_y']]

allowed_df.columns = ['date_game','team_name','opposing_team','Team TS%','Team eFG%','Team ORtg','Opp TS%', 'Opp eFG%', 'Opp ORtg']

In [33]:
export_csv = allowed_df.to_csv (r'C:\Users\jeromerufin\Desktop\Metis\advstats_16_17.csv') #Don't forget to add '.csv' at the end of the path